In [0]:
# Get Learning Cruves

The goal of this jupyter notebook is to get the learning curves accuracies in tran, validation an test for the baseline model and MLNT-teacher and MLNT-student neural networks for each epoch.

It is intended to be used in google colab.
A folder named *learning_curves* needs to be created in order to store the results.
It retrieves data from the checkpoint folder

SyntaxError: ignored

In [0]:
!pip install wandb

     |████████████████████████████████| 1.4MB 2.8MB/s 
     |████████████████████████████████| 102kB 8.4MB/s 
     |████████████████████████████████| 102kB 7.8MB/s 
     |████████████████████████████████| 112kB 19.5MB/s 
     |████████████████████████████████| 460kB 17.0MB/s 
     |████████████████████████████████| 71kB 7.2MB/s 
     |████████████████████████████████| 71kB 7.9MB/s 
  Created wheel for watchdog: filename=watchdog-0.10.2-cp36-none-any.whl size=73605 sha256=2dc90e8894301e00e1aaece3d544fa2c9ec27bb21eee429e658e61a20bcf72b4
  Stored in directory: /root/.cache/pip/wheels/bc/ed/6c/028dea90d31b359cd2a7c8b0da4db80e41d24a59614154072e
  Created wheel for gql: filename=gql-0.2.0-cp36-none-any.whl size=7630 sha256=9fa62f255256131721bbb2f01ea8dfb9786f92b6c6920c216b284a6564e48b57
  Stored in directory: /root/.cache/pip/wheels/ce/0e/7b/58a8a5268655b3ad74feef5aa97946f0addafb3cbb6bd2da23
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp36-none-any.whl size=6489 sha256=750f

In [0]:
from google.colab import drive
drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


In [0]:
import wandb
import sys
sys.path.append('/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar')

# Pytorch libraries
import torch
import torch.nn.functional as F
import torch.backends.cudnn as cudnn

# Internal files
import config
import dataloader
import models
# from baseline import get_model, save_checkpoint

import math
import os
import sys
import time
from collections import OrderedDict
import random

import pandas as pd




noise file noisy_label_kv.txt generated with noise: 0.5



<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [0]:
def get_model():

    # Get model from config
    if config.model == "resnet18":
        model = models.resnet18(pretrained=config.pretrained)
    elif config.model == "resnet34":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnet50':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "resnet101":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "resnet152":
        model = models.resnet152(pretrained=config.pretrained)
    elif config.model == "resnext50_32x4d":
        model = models.resnet34(pretrained=config.pretrained)
    elif config.model == 'resnext101_32x8d':
        model = models.resnet50(pretrained=config.pretrained)
    elif config.model == "wide_resnet50_2":
        model = models.resnet101(pretrained=config.pretrained)
    elif config.model == "wide_resnet101_2":
        model = models.resnet152(pretrained=config.pretrained)
    else:
        raise ValueError('%s not supported'.format(config.model))

    # Initialize fc layer
    (in_features, out_features) = model.fc.in_features, model.fc.out_features
    model.fc = torch.nn.Linear(in_features, out_features)
    return model

In [0]:
# Get the original_dataset
loader = dataloader.KeyDataLoader()
train_loader, valid_loader, test_loader = loader.run()

# Hyper Parameter settings
random.seed(config.seed)
# torch.cuda.set_device(config.gpuid)
torch.manual_seed(config.seed)
torch.cuda.manual_seed_all(config.seed)
use_cuda = torch.cuda.is_available()

# Networks setup
print('\nModel setup')
print('| Building network: {}'.format(config.model))
net = get_model()

if use_cuda:
  net.cuda()


Model setup
| Building network: resnet34


/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:122: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight, mode='fan_out')
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:124: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.weight, 1)
/content/drive/My Drive/Colab_Notebooks/git/PFM_Noisy_Labels/MLNT_cifar/models/resnet.py:125: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  nn.init.constant(m.bias, 0)


In [0]:
def load_checkpoint(epoch):
  # checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.checkpoint + "_" + str(epoch) + '.pth.tar')
  checkpoint = torch.load(config.drive_dir + '/checkpoint/' + config.id + "_student_" + str(epoch) + '.pth.tar')
  net.load_state_dict(checkpoint['state_dict'])

In [0]:
def test(loader):
    net.eval()
    # test_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(loader):
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        with torch.no_grad():
            outputs = net(inputs).cuda()

        # test_loss += loss.data.item()
        _, predicted = torch.max(outputs.data, 1)
        total += targets.size(0)
        correct += predicted.eq(targets.data).cpu().sum()

    # Grab validation results
    test_acc = 100. * correct/total

    return(round(float(test_acc),2))

In [0]:
def get_accy_list(max_epoch, loader):
  max_epoch = 101

  accy_list = []
  for epoch in range(1,101):
    load_checkpoint(epoch)
    accy_list.append(test(loader))
  return(accy_list)
    # print(accy_list)

In [10]:
print("computing test list\n")
test_list  = get_accy_list(config.num_epochs, test_loader)
print(test_list)
print("computing val list\n")
valid_list = get_accy_list(config.num_epochs, valid_loader)
print(valid_list)
print("computing train list\n")
train_list = get_accy_list(config.num_epochs, train_loader)

df = pd.DataFrame()
df["epoch"] = range(1,101)
df["test"] = test_list
df["valid"] = valid_list
df["train"] = train_list

computing test list

[10.46, 12.29, 14.38, 15.22, 13.29, 19.98, 23.02, 19.57, 27.15, 29.0, 25.9, 33.19, 35.67, 36.58, 41.95, 36.31, 44.11, 39.09, 39.64, 43.2, 51.54, 46.62, 51.89, 48.52, 52.63, 55.3, 53.76, 43.82, 52.72, 54.38, 59.86, 53.76, 55.62, 61.11, 62.39, 63.46, 59.24, 62.08, 55.05, 59.72, 62.6, 62.22, 63.59, 65.75, 48.0, 57.12, 60.7, 44.3, 35.38, 43.81, 52.61, 54.84, 46.44, 53.52, 58.14, 57.17, 54.27, 54.7, 60.6, 53.3, 47.21, 62.55, 56.01, 60.36, 62.98, 64.46, 62.91, 62.15, 55.22, 58.2, 64.34, 62.02, 62.6, 57.23, 60.98, 62.1, 65.88, 59.81, 59.35, 68.52, 67.07, 68.65, 64.82, 61.57, 62.61, 60.81, 60.23, 58.02, 55.36, 58.53, 54.8, 49.23, 44.24, 51.44, 46.84, 46.35, 53.27, 47.62, 43.67, 43.93]
computing val list

[10.56, 11.46, 14.2, 14.12, 12.66, 19.68, 22.28, 18.74, 26.28, 29.34, 26.44, 33.5, 36.32, 36.2, 41.16, 36.28, 44.66, 39.44, 39.98, 43.96, 51.78, 47.3, 53.18, 48.58, 54.24, 56.66, 55.48, 44.64, 54.26, 55.16, 60.38, 54.88, 56.44, 61.78, 63.26, 64.44, 60.32, 63.7, 55.06, 60.1

In [11]:
df = pd.DataFrame()
df["epoch"] = range(1,101)
df["test"] = test_list
df["valid"] = valid_list
df["train"] = train_list

df

,epoch,test,valid,train
0,1,10.46,10.56,9.90
1,2,12.29,11.46,10.92
2,3,14.38,14.20,11.96
3,4,15.22,14.12,11.72
4,5,13.29,12.66,10.95
...,...,...,...,...
95,96,46.35,47.30,29.21
96,97,53.27,54.24,33.06
97,98,47.62,48.18,29.76
98,99,43.67,44.68,28.35


In [0]:
# clean_kv = config.drive_dir +"/learning_curves/"+ config.checkpoint+".csv"
clean_kv = config.drive_dir +"/learning_curves/"+ config.id+"_student.csv"
# clean_kv = "/content/drive/My Drive/Colab_Notebooks/pfm/data/asdf.txt"
df.to_csv(clean_kv, sep=';', index=False,header = True)